In [29]:
import sys
sys.path.append("../../../../ns3_sem")
import sem
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
ns_path = '../../../../../ns3-dev-hofailure_RACH'
script = 'lena_hofailure'
campaign_dir = 'rngrun1'
campaign = sem.CampaignManager.new(ns_path, script, campaign_dir, overwrite=True)
print(campaign)

--- Campaign info ---
script: lena_hofailure
params: {'simTime': '50', 'nBlocks': '1', 'nMacroEnbSites': '1', 'macroUeDensity': '1e-05', 'homeEnbDeploymentRatio': '0.2', 'homeEnbActivationRatio': '0.5', 'homeUesHomeEnbRatio': '1', 'macroEnbDlEarfcn': '100', 'homeEnbDlEarfcn': '100', 'macroEnbBandwidth': '25', 'homeEnbBandwidth': '25', 'generateRem': 'false', 'epc': 'true', 'useUdp': 'true', 'outdoorUeMinSpeed': '0', 'outdoorUeMaxSpeed': '0', 'handoverAlgorithm': 'A3', 'Hysteresis': 3.0, 'TimeToTrigger': 256.0, 'ServingCellThreshold': 20.0, 'NeighbourCellOffset': 5.0, 'SrsPeriodicity': 80.0, 'epcDl': 'true', 'epcUl': 'true', 'fadingTrace': '', 'homeEnbTxPowerDbm': '20', 'interSiteDistance': '500', 'macroEnbTxPowerDbm': '43', 'nMacroEnbSitesX': '1', 'numBearersPerUe': '1', 'remRbId': '-1', 'srsPeriodicity': '80'}
HEAD: c6d9a9f8115a3ba83d26aafe492241a40bf89c29
Runner type: <class 'sem.parallelrunner.ParallelRunner'>
-----------


In [30]:
import numpy as np
param_combination = {'simTime':5000, 
                     'nBlocks': 4, #list([1,2,3,4,5,6,7,8,9,10]),#round(4*nBlocks*0.1)
                     'nMacroEnbSites': 1, #list([1,2,3,4,5,6,7,8,9,10]),
                     'nMacroEnbSitesX': 1,
                     'macroUeDensity': 0.00001,#1000000 [p * 0.5 for p in range(0.0001, 0.01)]
                     'homeEnbDeploymentRatio': 1.0,
                     'homeEnbActivationRatio': 1.0,
                     'homeUesHomeEnbRatio': 1.0,
                     'outdoorUeMinSpeed': 1.0,
                     'outdoorUeMaxSpeed': 1.0,
                     'handoverAlgorithm': 'A2A4',
                     'ServingCellThreshold':[j for j in np.arange(0.0, 5.0, 0.5)],
                     'NeighbourCellOffset':[i for i in np.arange(0.0, 5.0, 0.5)],
                     #'Hysteresis': [j * 1.0 for j in np.arange(0.0, 3.2, 0.2)],
                     #'TimeToTrigger':  list([0.0, 40.0, 64.0, 80.0, 100.0, 128.0, 160.0, 256.0, 320.0, 480.0, 512.0]),
                     'SrsPeriodicity': 80,
                     'generateRem': 0,
                     'RngRun': [1]
                    }

In [31]:
campaign.run_missing_simulations(param_combination, stop_on_errors=False)

Running simulations:   0%|                                                                               | 0/100 [02:12<?, ?simulation/s]


KeyboardInterrupt: 

In [20]:
def get_simple_result(result_rngrun1):
    if result_rngrun1['output']['stderr'] != "":
        num_HOs = -1
        num_RLFs = -1
        num_HFMRs = -1
        num_HFNPs = -1
        num_HFJs = -1
        num_HFLs = -1
    else:
        split_result_rngrun1 = result_rngrun1['output']['stdout'].split("\n")
        num_HOs = 0
        num_RLFs = 0
        num_HFMRs = 0
        num_HFNPs = 0
        num_HFJs = 0
        num_HFLs = 0
        for line in split_result_rngrun1:
            event = line.split(":",1)[0]
            if event == "Handover_start":
                num_HOs += 1
            elif event == "Handover_Failure_NoPreamble":
                num_HFNPs += 1
            elif event == "Handover_Failure_JoiningTimeout":
                num_HFJs += 1
            elif event == "Handover_Failure_MaxRach":
                num_HFMRs += 1
            elif event == "Handover_Failure_LeavingTimeout":
                num_HFLs += 1
            elif event == "Radio_link_failure_detected":
                num_RLFs += 1
    return [num_HOs, num_HFJs]

In [21]:
results_rngrun1 = campaign.get_results_as_dataframe(get_simple_result, 
                                            ['num_HOs','num_HFJs'], 
                                            params = param_combination, drop_constant_columns=True)
print(results_rngrun1)

    ServingCellThreshold  NeighbourCellOffset
0                    0.0                  0.0
1                    0.0                  2.0
2                    0.0                  4.0
3                    0.0                  6.0
4                    0.0                  8.0
..                   ...                  ...
95                  18.0                 10.0
96                  18.0                 12.0
97                  18.0                 14.0
98                  18.0                 16.0
99                  18.0                 18.0

[100 rows x 2 columns]


In [ ]:
results_Hys1 = results_rngrun_ave[(results_rngrun_ave['TimeToTrigger'] == 256.0)]
plt.rcParams.update({'font.size':14})
fig = plt.figure(figsize=(7, 5))
ax = plt.axes()
ax.plot(results_Hys1['Hysteresis'], results_Hys1['num_HOs'], color = 'blue')
ax.plot(results_Hys1['Hysteresis'], results_Hys1['num_HFJs'], color = 'red')
#ax.plot(results_Hys['Hysteresis'], results_Hys['num_RLFs'], color = 'black')
ax.grid()
ax.set_xlabel('Hysteresis')
ax.set_ylabel('Number of HOs and HFJs')
ax.set_title('TimeToTrigger = 256 ms, speed = 1 m/s')
ax.legend(['num_HOs', 'num_HFJs','num_RLFs'])

In [28]:
j = [j for j in np.arange(0.0, 5.0, 0.5)]
j

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2000000000000002,
 1.3,
 1.4000000000000001,
 1.5,
 1.6,
 1.7000000000000002,
 1.8,
 1.9000000000000001,
 2.0,
 2.1,
 2.2,
 2.3000000000000003,
 2.4000000000000004,
 2.5,
 2.6,
 2.7,
 2.8000000000000003,
 2.9000000000000004]